In [ ]:
pip install pandas scikit-learn nltk bnlp

In [ ]:
pip install bnlp_toolkit

In [ ]:
pip install -U bnlp_toolkit

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
from bnlp import NLTKTokenizer

In [ ]:
import pandas as pd

# Specify the file path of the XLSX file
file_path = "/content/dataset2.xlsx"

# Read the XLSX file into a pandas DataFrame
try:
    data = pd.read_excel(file_path)
except UnicodeDecodeError:
    print("Error: Failed to decode XLSX file. Please check the encoding.")
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")

# Now you can work with the 'data' DataFrame containing the contents of the XLSX file.


In [ ]:
class_column = 'Class'
unique_classes = data[class_column].unique()
num_classes = len(unique_classes)

print(f"Number of unique classes: {num_classes}")
print("Unique classes:", unique_classes)

Number of unique classes: 2
Unique classes: ['original' 'AI']


In [ ]:
class_column = 'Class'

# Count the occurrences of each class
class_counts = data[class_column].value_counts()

print("Class distribution:")
print(class_counts)

Class distribution:
AI          129
original    124
Name: Class, dtype: int64


In [ ]:
data

,Text,Class
0,I want to understand various states of substan...,original
1,I aim to explore the various states of substan...,AI
2,Subtances consist of a great many atoms. An at...,original
3,"Substances are composed of numerous atoms, whi...",AI
4,The molecular dynamics method deals with atoms...,original
...,...,...
248,I implement WebSocket Module to CVE-Client Pro...,original
249,"Then, They can send & recieve some information...",original
250,"This can be run various device (Windows, Mac, ...",original
251,I suceed render 3D panoramic picture on WebBro...,original


In [ ]:
pip install tensorflow keras scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.utils import to_categorical


# Split the dataset into features (X) and labels (y)
X = data['Text']
y = data['Class']

# Encode the target labels to numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to make them of the same length
max_sequence_length = 1000  # You can adjust this value based on your dataset and text length
X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_sequence_length)

# Convert the target labels to one-hot encoding for 2 classes
y_train_onehot = to_categorical(y_train, num_classes=2)
y_test_onehot = to_categorical(y_test, num_classes=2)


In [ ]:
# Create the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(Dense(units=2, activation='softmax'))  # For 3-class classification

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_padded, y_train_onehot, batch_size=32, epochs=25, validation_data=(X_test_padded, y_test_onehot))

# Evaluate the model
_, accuracy = model.evaluate(X_test_padded, y_test_onehot)
print("Accuracy:", accuracy)

Epoch 1/25
7/7 [==============================] - 6s 417ms/step - loss: 0.6924 - accuracy: 0.5099 - val_loss: 0.7060 - val_accuracy: 0.3922
Epoch 2/25
7/7 [==============================] - 3s 413ms/step - loss: 0.6777 - accuracy: 0.5396 - val_loss: 0.7267 - val_accuracy: 0.3922
Epoch 3/25
7/7 [==============================] - 5s 678ms/step - loss: 0.6792 - accuracy: 0.6832 - val_loss: 4.3240 - val_accuracy: 0.3922
Epoch 4/25
7/7 [==============================] - 6s 846ms/step - loss: 1.5411 - accuracy: 0.6931 - val_loss: 0.6643 - val_accuracy: 0.6863
Epoch 5/25
7/7 [==============================] - 4s 550ms/step - loss: 0.5925 - accuracy: 0.9455 - val_loss: 0.6694 - val_accuracy: 0.6078
Epoch 6/25
7/7 [==============================] - 6s 935ms/step - loss: 0.5890 - accuracy: 0.9109 - val_loss: 0.6705 - val_accuracy: 0.6078
Epoch 7/25
7/7 [==============================] - 3s 561ms/step - loss: 0.5541 - accuracy: 0.9307 - val_loss: 0.6676 - val_accuracy: 0.5882
Epoch 8/25
7/7 [====

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Get the model predictions on the test set
y_pred_prob = model.predict(X_test_padded)
y_pred_labels = np.argmax(y_pred_prob, axis=1)

# Invert one-hot encoding for y_test
y_test_labels = np.argmax(y_test_onehot, axis=1)

# Calculate F1 score
f1 = f1_score(y_test_labels, y_pred_labels, average='macro')
print("F1 Score:", f1)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test_labels, y_pred_labels)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate ROC-AUC curve
roc_auc = roc_auc_score(y_test_onehot, y_pred_prob, average='macro')
print("ROC-AUC Score:", roc_auc)

# Plot ROC curve for each class
fpr = dict()
tpr = dict()
roc_auc_per_class = dict()



2/2 [==============================] - 0s 26ms/step
F1 Score: 0.7386677177769019
Confusion Matrix:
[[15  5]
 [ 8 23]]
ROC-AUC Score: 0.7629032258064515


In [ ]:
# Make predictions on the user input
user_text = input("Enter text: ")
user_text_seq = tokenizer.texts_to_sequences([user_text])
user_text_padded = pad_sequences(user_text_seq, maxlen=max_sequence_length)
user_prediction = model.predict(user_text_padded)
user_prediction_label = np.argmax(user_prediction)  # Get the class index with the highest probability
user_prediction_class = label_encoder.inverse_transform([user_prediction_label])[0]

print("Predicted :", user_prediction_class)

Enter text: Furthermore, I observed that even a minor fluctuation introduced to a metastable square lattice state caused it to transition towards the more favorable triangular lattice arrangement. These insights have enabled me to develop an intuitive comprehension of substance states, their structural transformations, and the shifts in their thermodynamic properties.
1/1 [==============================] - 0s 54ms/step
Predicted : AI


In [ ]:
pip install tensorflow tensorflow-datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder

# Split the dataset into features (X) and labels (y)
X = data['Text']
y = data['Class']

# Use LabelEncoder to convert labels to integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize the text and convert them into input features
input_ids = []
attention_masks = []

for text in X:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert lists to tensors
input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)

# Convert to NumPy arrays
X_np = np.array(input_ids)
y_np = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

# Create TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Batch the data and shuffle
batch_size = 32
train_dataset = train_dataset.shuffle(len(X_train)).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

# Load pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Set up optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model with hyperparameter tuning
epochs = 5
history = model.fit(train_dataset, epochs=epochs, validation_data=test_dataset)

# Evaluation on test data
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy*100:.2f}%")


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
7/7 [==============================] - 71s 1s/step - loss: 1.3827 - accuracy: 0.3960 - val_loss: 1.1211 - val_accuracy: 0.6275
Epoch 2/5
7/7 [==============================] - 6s 801ms/step - loss: 1.0255 - accuracy: 0.6139 - val_loss: 0.8806 - val_accuracy: 0.6471
Epoch 3/5
7/7 [==============================] - 7s 960ms/step - loss: 0.7869 - accuracy: 0.8317 - val_loss: 0.7592 - val_accuracy: 0.8039
Epoch 4/5
7/7 [==============================] - 6s 929ms/step - loss: 0.5941 - accuracy: 0.9158 - val_loss: 0.5834 - val_accuracy: 0.8627
Epoch 5/5
2/2 [==============================] - 1s 173ms/step - loss: 0.5032 - accuracy: 0.8824
Test Accuracy: 88.24%
